In [9]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd

from extract_features import extract_features
from read_conllu_to_csv import read_conllu_write_csv
from read22 import split_gold_column

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import numpy as np

In [10]:
train_file = "../UP-1.0/input/en_ewt-up-train.conllu"
dev_file = "../UP-1.0/input/en_ewt-up-dev.conllu"
test_file = "../UP-1.0/input/en_ewt-up-test.conllu"

out_train_file = '../UP-1.0/output/read_conllu_train.csv'
out_test_file = '../UP-1.0/output/read_conllu_test.csv'
out_dev_file = '../UP-1.0/output/read_conllu_dev.csv'

feat_train_file = '../UP-1.0/features/extract_features_out_train.csv'
feat_test_file = '../UP-1.0/features/extract_features_out_test.csv'
feat_dev_file = '../UP-1.0/features/extract_features_out_dev.csv'

In [3]:
read_conllu_write_csv(dev_file, out_dev_file)
read_conllu_write_csv(test_file, out_test_file)
# read_conllu_write_csv(train_file, out_train_file)

File dev.csv produced!
File test.csv produced!


,0,1,2,3,4,5,6,7,8,9,10,11
0,1,What,what,PRON,WP,PronType=Int,0,root,0:root,_,_,_
1,2,if,if,SCONJ,IN,_,4,mark,4:mark,_,_,_
2,3,Google,Google,PROPN,NNP,Number=Sing,4,nsubj,4:nsubj,_,_,ARG1
3,4,Morphed,morph,VERB,VBD,Mood=Ind|Tense=Past|VerbForm=Fin,1,advcl,1:advcl:if,_,morph.01,V
4,5,Into,into,ADP,IN,_,6,case,6:case,_,_,_
...,...,...,...,...,...,...,...,...,...,...,...,...
100923,16,suggesting,suggest,VERB,VBG,VerbForm=Ger,7,conj,5:advcl:in|7:conj:and,_,suggest.01,_
100924,17,exercises,exercise,NOUN,NNS,Number=Plur,16,obj,16:obj,_,exercise.02,ARG1
100925,18,to,to,PART,TO,_,19,mark,19:mark,_,_,_
100926,19,use,use,VERB,VB,VerbForm=Inf,17,acl,17:acl:to,SpaceAfter=No,use.01,V


In [4]:
extract_features(out_dev_file, feat_dev_file)
# extract_features(out_train_file, feat_train_file)
extract_features(out_test_file, feat_test_file)

exctacted voice
exctacted voice


In [90]:
vectorizer_id = DictVectorizer()

In [91]:
def add_indetification_labels(df):
    # df = pd.read_csv( '../UP-1.0/features/extract_features_out_dev.csv', sep='\t')
    preds, args = split_gold_column(df.Label.tolist())
    ident_args = [1 if a != '_' else 0 for a in args]
    df['Identify args'] = ident_args
    return df

In [92]:
df_train = pd.read_csv(feat_train_file, sep='\t')
df_dev = pd.read_csv(feat_dev_file, sep='\t')
df_test = pd.read_csv(feat_test_file, sep='\t')

df_train = add_indetification_labels(df_train)
df_dev = add_indetification_labels(df_dev)
df_test = add_indetification_labels(df_test)

df_train = df_train[df_train['Token'].notna()]
df_dev = df_dev[df_dev['Token'].notna()]
df_test = df_test[df_test['Token'].notna()]

df_train.Label = df_train['Label'].fillna(value='None_placeholder')
df_dev.Label = df_dev['Label'].fillna(value='None_placeholder')
df_test.Label = df_test['Label'].fillna(value='None_placeholder')

df_train.Head = df_train.Head.fillna(value='_')
df_dev.Head = df_dev.Head.fillna(value='_')
df_test.Head = df_test.Head.fillna(value='_')

df_train['Morphological Feature'] = df_train['Morphological Feature'].fillna(value='_')
df_dev['Morphological Feature'] = df_dev['Morphological Feature'].fillna(value='_')
df_test['Morphological Feature'] = df_test['Morphological Feature'].fillna(value='_')

# these guys are None
df_train.drop(columns=['Entity_type','E_DEP', 'voice', 'Gold'], inplace=True)
df_dev.drop(columns=['Entity_type','E_DEP', 'voice', 'Gold'], inplace=True)
df_test.drop(columns=['Entity_type','E_DEP', 'voice', 'Gold'], inplace=True)

C:\Users\snipercapt\AppData\Local\Temp\ipykernel_15872\923600145.py:1: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(feat_train_file, sep='\t')
C:\Users\snipercapt\AppData\Local\Temp\ipykernel_15872\923600145.py:2: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dev = pd.read_csv(feat_dev_file, sep='\t')
C:\Users\snipercapt\AppData\Local\Temp\ipykernel_15872\923600145.py:3: DtypeWarning: Columns (10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(feat_test_file, sep='\t')


## Identification model

In [94]:
X_train = df_train.drop('Identify args', axis='columns')
X_dev = df_dev.drop('Identify args', axis='columns')
X_test = df_test.drop('Identify args', axis='columns')

y_train_id = df_train['Identify args']
y_test_id = df_test['Identify args']
y_dev_id = df_dev['Identify args']

X_train_vec = vectorizer_id.fit_transform(X_train.to_dict('records'))
X_dev_vec = vectorizer_id.transform(X_dev.to_dict('records'))
X_test_vec = vectorizer_id.transform(X_test.to_dict('records'))

In [95]:
model_id = LogisticRegression(solver='liblinear', max_iter=1000)

model_id.fit(X_train_vec, y_train_id)

LogisticRegression(max_iter=1000, solver='liblinear')

In [96]:
y_pred_id = model_id.predict(X_dev_vec)

In [97]:
print("METRICS FOR ARG IDENTIFICATION")
print('precision', precision_score(y_dev_id, y_pred_id, average='macro'), sep='\t')
print('recall', recall_score(y_dev_id, y_pred_id, average='macro'), sep='\t')
print('f1', f1_score(y_dev_id, y_pred_id, average='macro'), sep='\t')

METRICS
precision	1.0
recall	1.0
f1	1.0


In [98]:
print(classification_report(y_dev_id, y_pred_id))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     98006
           1       1.00      1.00      1.00      8017

    accuracy                           1.00    106023
   macro avg       1.00      1.00      1.00    106023
weighted avg       1.00      1.00      1.00    106023



## Classification model

In [99]:
vectorizer = DictVectorizer()

In [100]:
df_dev_class = df_dev[df_dev['Identify args']==1]
df_train_class = df_train[df_train['Identify args']==1]
df_test_class = df_test[df_test['Identify args']==1]

# df_test_class.drop('Identify args', axis='columns', inplace=True)
# df_train_class.drop('Identify args', axis='columns', inplace=True)
# df_dev_class.drop('Identify args', axis='columns', inplace=True)

df_train_class.head()

,Token,PoS,Lemma,Dependency,Head,Head_POS,Morphological Feature,Path to head texts,Path to head POS,Label,Identify args
0,1,NUM,1,nummod,Zaman,PROPN,NumType=Card,['1'],['NUM'],11,1
3,Zaman,PROPN,Zaman,ROOT,Zaman,PROPN,Number=Sing,[],[],None_placeholder,1
39,causing,VERB,cause,ROOT,causing,VERB,Aspect=Prog|Tense=Pres|VerbForm=Part,[],[],None_placeholder,1
57,causing,VERB,cause,ROOT,causing,VERB,Aspect=Prog|Tense=Pres|VerbForm=Part,[],[],None_placeholder,1
75,causing,VERB,cause,ROOT,causing,VERB,Aspect=Prog|Tense=Pres|VerbForm=Part,[],[],None_placeholder,1


In [101]:
X_train = df_train_class.drop('Label', axis='columns')
X_dev = df_dev_class.drop('Label', axis='columns')
X_test = df_test_class.drop('Label', axis='columns')

y_train = df_train_class['Label']
y_test = df_test_class['Label']
y_dev = df_dev_class['Label']

X_train_vec = vectorizer.fit_transform(X_train.to_dict('records'))
X_dev_vec = vectorizer.transform(X_dev.to_dict('records'))
X_test_vec = vectorizer.transform(X_test.to_dict('records'))

In [102]:
model = LogisticRegression(solver='liblinear', max_iter=1000)

model.fit(X_train_vec, y_train)

LogisticRegression(max_iter=1000, solver='liblinear')

In [103]:
y_pred = model.predict(X_dev_vec)

In [104]:
print("METRICS FOR ARG CLASSIFICATION")
print('precision', precision_score(y_dev, y_pred, average='macro'), sep='\t')
print('recall', recall_score(y_dev, y_pred, average='macro'), sep='\t')
print('f1', f1_score(y_dev, y_pred, average='macro'), sep='\t')

METRICS
precision	0.3890348943414966
recall	0.12439580281723019
f1	0.14740047019998695


c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [105]:
print(classification_report(y_dev, y_pred))

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_cla

                  precision    recall  f1-score   support

              11       1.00      0.14      0.24        22
            ARG0       0.52      0.18      0.27       766
            ARG1       0.42      0.76      0.54      2756
        ARG1-DSP       0.00      0.00      0.00         0
            ARG2       0.44      0.47      0.45      1673
            ARG3       0.00      0.00      0.00       119
            ARG4       0.00      0.00      0.00        32
        ARGM-ADJ       0.83      0.02      0.05       201
        ARGM-ADV       0.43      0.29      0.35       277
        ARGM-CAU       1.00      0.05      0.10        78
        ARGM-COM       0.00      0.00      0.00        60
        ARGM-DIR       0.00      0.00      0.00       303
        ARGM-DIS       0.91      0.61      0.73       280
        ARGM-EXT       1.00      0.09      0.16        47
        ARGM-GOL       0.00      0.00      0.00        24
        ARGM-LOC       0.00      0.00      0.00       205
        ARGM-

c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
